<b>Install necessary Python updates as well as import AutoML library</b>

In [ ]:
! pip install --upgrade pip
! pip install --upgrade google-cloud-automl

from google.cloud import automl_v1beta1 as automl

<b>Create necessary variables</b>

In [ ]:
PROJECT_ID = 'auto-modeling-v402'
REGION = 'us-central1'

RAW_SB = 'auto-modeling-v402_raw_data'

ZILLOW_PATH = 'bq://' + PROJECT_ID + '.r_dataset.zillow_data'
NYC_PATH = 'bq://' + PROJECT_ID + '.r_dataset.nyc_taxi_data'
ZILLOW_DATASET_DN = 'zillow_dataset'
NYC_DATASET_DN = 'NYC_taxi_dataset'
ZILLOW_MODEL = 'zillow_model_v1'
NYC_MODEL = 'NYC_taxi_model_v1'
ZILLOW_TARGET_COLUMN = 'logerror'
NYC_TARGET_COLUMN = 'fare_amount'

TRAIN_BUDGET = 2000

<b>Create datasets in AutoML</b>

In [ ]:
client = automl.TablesClient(project=PROJECT_ID, region=REGION)

dataset = client.create_dataset(ZILLOW_DATASET_DN)
dataset = client.create_dataset(NYC_DATASET_DN)

<b>Import data into AutoML datasets from BigQuery</b>

In [ ]:
if ZILLOW_PATH.startswith("bq"):
    response = client.import_data(
        dataset_display_name=ZILLOW_DATASET_DN,
        bigquery_input_uri=ZILLOW_PATH
    )
    
if NYC_PATH.startswith("bq"):
    response = client.import_data(
        dataset_display_name=NYC_DATASET_DN,
        bigquery_input_uri=NYC_PATH
    )

<b>Set target columns, update nullable columns, and train models</b>

In [ ]:
client.set_target_column(
    dataset_display_name=ZILLOW_DATASET_DN,
    column_spec_display_name=ZILLOW_TARGET_COLUMN
)

response = client.create_model(
    ZILLOW_MODEL,
    train_budget_milli_node_hours=TRAIN_BUDGET,
    dataset_display_name=ZILLOW_DATASET_DN,
    optimization_objective='MAE',
    exclude_column_spec_names=[ZILLOW_TARGET_COLUMN]
)

In [ ]:
client.set_target_column(
    dataset_display_name=NYC_DATASET_DN,
    column_spec_display_name=NYC_TARGET_COLUMN
)

client.update_column_spec(
    dataset_display_name=NYC_DATASET_DN,
    column_spec_display_name='dropoff_latitude',
    nullable=True
)

client.update_column_spec(
    dataset_display_name=NYC_DATASET_DN,
    column_spec_display_name='dropoff_longitude',
    nullable=True
)

response = client.create_model(
    NYC_MODEL,
    train_budget_milli_node_hours=TRAIN_BUDGET,
    dataset_display_name=NYC_DATASET_DN,
    exclude_column_spec_names=[NYC_TARGET_COLUMN]
)